# Homework 2

The goal of this homework is to get familiar with MLflow.

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments][conda-env], and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

[conda-env]: https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs

In [1]:
!mlflow --version

mlflow, version 1.26.1


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset.

Download the data for January, February and March 2021 in parquet format from [here][green-taxi-trip-data].

Use the script `preprocess_data.py` located in the folder [homework][homework-repo] to preprocess the data.

The script will

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2021 data),
* save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

[green-taxi-trip-data]: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
[homework-repo]: https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework

In [2]:
from pathlib import Path
import requests

def download_nyc_green_taxi_trip_data(year_month: str, save_dir: str) -> str:
    fname = f'green_tripdata_{year_month}.parquet'
    
    if not Path(f'{save_dir}/{fname}').exists():
        r = requests.get(f'https://nyc-tlc.s3.amazonaws.com/trip+data/{fname}')
    
        with open(f'{save_dir}/{fname}', 'wb') as fout:
            for chunk in r.iter_content(chunk_size=1024):
                fout.write(chunk)
            
    return f'{save_dir}/{fname}'

In [3]:
DATA_DIR = '.'

for year_month in ['2021-01', '2021-02', '2021-03']:
    download_nyc_green_taxi_trip_data(year_month, DATA_DIR)

In [4]:
!python homework2/preprocess_data.py --raw_data_path . --dest_path ./output

In [5]:
items_in_output_dir = len(list(Path('./output').iterdir()))
print(f'There are {items_in_output_dir} files in ./output')

There are 4 files in ./output


## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

* 19
* 17
* 10
* 20

In [6]:
!python homework2/train.py --data_path ./output

2022/05/30 21:37:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\tools\python\envs\mlops-zoomcamp\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments()
last_experiment = experiments[-1]
run_infos = client.list_run_infos(last_experiment.experiment_id)
last_run_info = run_infos[-1]
run = client.get_run(last_run_info.run_id)
run_params = run.data.params

print(f'Number of parameters automatically logged: {len(run_params)}')

Number of parameters automatically logged: 17
